In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
data=pd.read_csv("../input/records/records.csv")
all_text=data["transcription_text"]

In [3]:
data.shape

(35168, 16)

In [4]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punctuation
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    return s
start=time.time()
for i in tqdm(range(35168)):
    all_text[i] = process(all_text[i])
for i in tqdm(range(35168)):
    all_text[i] = " ".join(all_text[i])
end=time.time()
print(end-start)

100%|██████████| 35168/35168 [00:18<00:00, 1934.31it/s]

112.68326330184937


In [5]:
all_text

0        one hundred dollars . I never heard Bagby comp...
1        The Freedmens Bureau vs. B.M . Richardson Assa...
2        The Freedmens Bureau vs. B. M. Richardson , Co...
3        one hundred dollars , two suits of clothes wou...
4        questions , I gave answer , I whipped.— All I ...
                               ...                        
35163    648 Z Date.Name of Male.Place of Residence.Nam...
35164    650 Date . Name of Male . Name of Female . Pla...
35165    Sarah Brame Died June 25th 1847 . 1866 1847 De...
35166    Frances S. Benners Born August 30th 1795 Died ...
35167    State of North Carolina Bladen County Alfred ....
Name: transcription_text, Length: 35168, dtype: object

****

****

In [6]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 900.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.1-py3-none-any.whl size=125774 sha256=491dcbac65464930115d30f6bdcdf25797781751d53c208cab8e5322b9001ed9
  Stored in directory: /root/.cache/pip/wheels/58/27/2f/708b4f002c226e57b6243769da345c650633175c7634f93365
Successfully built sentence-transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.7.0
    Uninstalling huggingface-hub-0.7.0:
      Successfully uninstalled huggingface-hub-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cached-path 1.1.3 requires huggingface-hub<0.8.0,>=0.0.12, but you have huggingface-hub 0.8.1 which i

In [7]:
def num_of_element(clust_assign, clus_sent) :
    dic={}
    L=list(clust_assign)
    for i in range(len(clus_sent )):
        dic[i]=L.count(i)
    
    return dic
        

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(all_text)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1099 [00:00<?, ?it/s]

In [9]:
embeddings.shape

(35168, 384)

In [10]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

256.1831245422363


In [11]:
len(clustered_sentences)

309

In [12]:
cluster_assignment

array([ 58,  58,  58, ..., 125, 125, 186])

In [13]:
dic1=num_of_element(cluster_assignment,clustered_sentences)

In [14]:
dic1[0]

153

In [15]:
clustered_sentences[0][1]

'lawfull wages period herein Specified withold fail pay said Bonaparte Fulford check number aforesaid appropriate use benefit . This Fort Macon North Carolina twelvth day of November A.D. 1866 . Specification 2nd In Second Lieutenant James H. Griggs Thirty Seventh United States Colored Troops , Officer service of United States Acting Assistant Quartermaster Post of Fort Macon North Carolina Command one Hasty Monroe , citizen employ of United States first fifth day of July A.D. 1866 , 1 20 day of August A D 1866 inclusive compensation of thirty dollars per month , furnished Brevet Lieutenant Colonel T. D. Stubbs , Captain Assistant Quartermaster United States Volunteers , Cheif Quartermaster Military Command of North Carolina twelvth day of November A.D. 1866 , check number payable order Assistant Treasury New York sum of Twenty five dollars purpose of paying said Hasty Monroe lawful wages period herein specified withhold fail pay said Hasty Monroe check number'

****

# all_mpnet_base_v2 , threshold=3.5

In [16]:
model_1 = SentenceTransformer('all-mpnet-base-v2')
embeddingss = model_1.encode(all_text)
embeddingss.shape

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1099 [00:00<?, ?it/s]

(35168, 768)

In [17]:
embeddingss.shape

(35168, 768)

In [18]:
start=time.time()
# Normalize the embeddings to unit length
embeddingsqs = embeddingss /  np.linalg.norm(embeddingss, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddingss)
cluster_assignment_1 = clustering_model.labels_

clustered_sentences_1 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment_1):
    if cluster_id not in clustered_sentences_1:
        clustered_sentences_1[cluster_id] = []

    clustered_sentences_1[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

371.0782263278961


In [19]:
len(clustered_sentences_1)

245

In [20]:
dic_1=num_of_element(cluster_assignment_1,clustered_sentences_1)

In [21]:
cluster_assignment_1

array([ 53, 140, 140, ..., 179,  24, 147])

In [22]:
clustered_sentences_1[0][1]

'C North Carolina Columbus County This day came Calvin G. Gore , justice of peace said County . J Lloyd McKay sworn saith oath well acquainted Ben Simpson freedman , seen engaged riot row freedmen knocking brass knuckles , regards person of bad vicious character ; would believe oath . That Mr Richardsons employ two months , always faithful careful paying men employ . Signed , Sealed Sworn I certify Mr McKay gentleman of respectability . J. L. McKay Seal Calvin C. Gore JP'

****

# all-MiniLM-L6-v2 , threshold=2

In [23]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=2) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment2 = clustering_model.labels_

clustered_sentences2 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment2):
    if cluster_id not in clustered_sentences2:
        clustered_sentences2[cluster_id] = []

    clustered_sentences2[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

255.56904458999634


In [24]:
len(clustered_sentences2)

1008

In [25]:
dic2=num_of_element(cluster_assignment2,clustered_sentences2)

In [26]:
dic2[0]

307

****

# all_mpnet_base_v2 , threshold=2

In [27]:

start=time.time()
# Normalize the embeddings to unit length
embeddingsqs = embeddingss /  np.linalg.norm(embeddingss, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=2) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddingss)
cluster_assignment_2 = clustering_model.labels_

clustered_sentences_2 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment_2):
    if cluster_id not in clustered_sentences_2:
        clustered_sentences_2[cluster_id] = []

    clustered_sentences_2[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)


370.8983199596405


In [28]:
len(clustered_sentences_2)

752

****

# all-MiniLM-L6-v2 , threshold=3

In [29]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment3 = clustering_model.labels_

clustered_sentences3 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment3):
    if cluster_id not in clustered_sentences3:
        clustered_sentences3[cluster_id] = []

    clustered_sentences3[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

256.3824956417084


In [30]:
len(clustered_sentences3)

407

In [31]:
dic3=num_of_element(cluster_assignment3,clustered_sentences3)

In [32]:
dic3[0]

92

****

# all-MiniLM-L6-v2 , threshold=4

In [33]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=4) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment4 = clustering_model.labels_

clustered_sentences4 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment4):
    if cluster_id not in clustered_sentences4:
        clustered_sentences4[cluster_id] = []

    clustered_sentences4[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

259.92244243621826


In [34]:
len(clustered_sentences4)

237

In [35]:
dic4=num_of_element(cluster_assignment4,clustered_sentences4)

In [36]:
dic4[0]

134

****

In [37]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment5 = clustering_model.labels_

clustered_sentences5 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment5):
    if cluster_id not in clustered_sentences5:
        clustered_sentences5[cluster_id] = []

    clustered_sentences5[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

259.7984046936035


In [38]:
len(clustered_sentences5)

146

In [39]:
dic5=num_of_element(cluster_assignment5,clustered_sentences5)

In [40]:
dic5

{0: 172,
 1: 654,
 2: 500,
 3: 179,
 4: 365,
 5: 634,
 6: 287,
 7: 351,
 8: 218,
 9: 1071,
 10: 278,
 11: 325,
 12: 468,
 13: 514,
 14: 328,
 15: 250,
 16: 185,
 17: 366,
 18: 184,
 19: 490,
 20: 484,
 21: 192,
 22: 598,
 23: 265,
 24: 101,
 25: 147,
 26: 644,
 27: 404,
 28: 399,
 29: 166,
 30: 236,
 31: 61,
 32: 708,
 33: 184,
 34: 195,
 35: 148,
 36: 182,
 37: 247,
 38: 124,
 39: 391,
 40: 260,
 41: 324,
 42: 374,
 43: 461,
 44: 143,
 45: 197,
 46: 442,
 47: 126,
 48: 277,
 49: 481,
 50: 205,
 51: 300,
 52: 94,
 53: 93,
 54: 62,
 55: 238,
 56: 140,
 57: 148,
 58: 306,
 59: 193,
 60: 583,
 61: 361,
 62: 373,
 63: 196,
 64: 89,
 65: 122,
 66: 469,
 67: 134,
 68: 131,
 69: 225,
 70: 60,
 71: 136,
 72: 171,
 73: 541,
 74: 304,
 75: 208,
 76: 189,
 77: 265,
 78: 265,
 79: 443,
 80: 67,
 81: 221,
 82: 30,
 83: 193,
 84: 313,
 85: 110,
 86: 261,
 87: 112,
 88: 118,
 89: 553,
 90: 104,
 91: 145,
 92: 293,
 93: 40,
 94: 164,
 95: 203,
 96: 88,
 97: 67,
 98: 112,
 99: 208,
 100: 247,
 101: 379